# 场景布局生成

1. 先将问题翻译为英文。后续推理过程使用英文会具有更好的推理性能
2. 使用self-discover方式分析问题，识别问题中的关键信息，推导出符合要求的布局，并对布局合理性进行解释
3. 使用self-discover方式分析问题，选择符合要求的模型资产，并对选择进行解释。并保存下资产的路径。
4. 结合上面的布局和资产将布局格式化为JSON格式文本，同时将解释翻译回中文
4. 将JSON文件转化为usda格式文本



In [15]:
from langchain_community.chat_models import ChatOllama

# llm = ChatOllama(model="llama3:8b-instruct-q5_K_M", temperature=0)
llm = ChatOllama(model="llama3:8b-instruct-q5_K_M", temperature=0)

## 翻译 Agent

In [16]:
# Import things that are needed generically
from langchain.agents import AgentType, Tool, initialize_agent

In [17]:
tools = [
    Tool(
        name="Translation Agent",
        func=None,
        description="You are a translation agent, proficient in Chinese and English translation, when asked to translate, very useful.",
    )
]

In [18]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [20]:
agent.invoke(
    "流浪地球是发生在哪一年的故事？"
)



> Entering new AgentExecutor chain...
Let's get started.

Thought: The question is asking about the year in which the story "流浪地球" (The Wandering Earth) takes place. I think I can help with that!

Action: [Translation Agent]
Action Input: None
Observation: [Translation Agent] is not a valid tool, try one of [Translation Agent].
Thought:I apologize for the mistake! Let me retry.

Thought: The question is asking about the year in which the story "流浪地球" (The Wandering Earth) takes place. I think I can help with that!
Action: [Translation Agent]
Action Input: None
Observation: [Translation Agent] is not a valid tool, try one of [Translation Agent].
Thought:I apologize for the mistake earlier!

Let me retry again.

Thought: The question is asking about the year in which the story "流浪地球" (The Wandering Earth) takes place. I think I can help with that!
Action: Translation Agent
Action Input: None

NotImplementedError: Tool does not support sync

In [44]:
agent.run("OrcaStudio定位于?")



> Entering new AgentExecutor chain...
 我需要查询松应科技OrcaStudio的定位信息。
Action: 松应科技 OrcaStudio 问答系统
Action Input: OrcaStudio的定位是什么
Observation: 对不起，我不知道OrcaStudio的定位是什么。我无法从给定的文档中获取相关信息。
Thought: 我需要改用一般性知识来回答。
Final Answer: 松应科技的OrcaStudio可能是一个3D内容协作平台，但具体的市场定位没有直接信息。一般来说，这样的平台可能会定位在3D设计、建筑可视化、游戏开发或者教育培训等领域，提供一个让团队成员可以共同创作、编辑和分享3D内容的工具。要获取准确的定位描述，建议查阅官方文档或访问松应科技的官方网站获取最新信息。

> Finished chain.


'松应科技的OrcaStudio可能是一个3D内容协作平台，但具体的市场定位没有直接信息。一般来说，这样的平台可能会定位在3D设计、建筑可视化、游戏开发或者教育培训等领域，提供一个让团队成员可以共同创作、编辑和分享3D内容的工具。要获取准确的定位描述，建议查阅官方文档或访问松应科技的官方网站获取最新信息。'

## Use the Agent solely as a router

如果您打算使用代理作为路由器，并且只是想直接返回RetrievalQAChain的结果，您也可以设置`return_direct=True`。

注意，在上面的示例中，代理在查询RetrievalQAChain之后做了一些额外的工作。你可以避免这个，直接返回结果。

In [13]:
tools = [
    Tool(
        name="流浪地球 问答系统",
        func=state_of_union.run,
        description="当您需要回答有关最新流浪地球的问题时，非常有用。输入应该是一个完全形成的问题。",
        return_direct=True,
    ),
    Tool(
        name="松应科技 OrcaStudio 问答系统",
        func=ruff.run,
        description="当您需要回答有关松应科技 OrcaStudio（一种3D内容协作平台）的问题时非常有用。输入应该是一个完全形成的问题。",
        return_direct=True,
    ),
]

In [14]:
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [15]:
agent.run(
    "流浪地球是发生在哪一年的故事？"
)



> Entering new AgentExecutor chain...
 我需要调用流浪地球问答系统来查找这个故事的时间框架。
Action: 流浪地球 问答系统
Action Input: 故事《流浪地球》发生的具体年份是什么
Observation: 故事《流浪地球》并未提供具体的发生年份。它讲述的是在不远的未来，太阳老化引发的一系列事件，导致人类不得不启动“流浪地球”计划，逃离太阳系寻找新的居住星球。小说中的时间线主要参照了地球发动机运行和地球流浪的时间跨度，如500年的加速阶段等，而不是基于现实世界的年份。


> Finished chain.


'故事《流浪地球》并未提供具体的发生年份。它讲述的是在不远的未来，太阳老化引发的一系列事件，导致人类不得不启动“流浪地球”计划，逃离太阳系寻找新的居住星球。小说中的时间线主要参照了地球发动机运行和地球流浪的时间跨度，如500年的加速阶段等，而不是基于现实世界的年份。'

In [16]:
agent.run("松应科技3D内容协作平台如何降低协作成本？")



> Entering new AgentExecutor chain...
 我需要调用OrcaStudio的问答系统来回答关于其降低成本的具体方法。
Action: 松应科技 OrcaStudio 问答系统
Action Input: 如何通过OrcaStudio降低3D内容协作的成本

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1



Observation: OrcaStudio 是一个强大的工具，可以用于优化和降低成本的 3D 内容协作。虽然我无法提供具体的步骤（因为我是一个基于文本的模型，不能直接操作软件），但我可以给你一些一般性的建议来帮助你利用 OrcaStudio 实现这个目标：

1. **统一工作流程**：OrcaStudio 支持多种 3D 格式和工具，允许团队成员使用他们熟悉的工具集。这减少了学习新软件的时间，从而提高了效率。

2. **实时协作**：通过 OrcaStudio 的实时共享编辑功能，团队成员可以同时在同一个项目上工作，消除了版本控制的问题，减少因协同不当导致的错误和重做工作。

3. **自动化任务**：OrcaStudio 可能包含一些脚本或自动化工具，可以自动执行重复的任务，比如资源打包、优化或者预处理，这将节省大量手动操作的时间。

4. **精细权限管理**：通过设置不同成员的访问权限，你可以确保只有授权人员才能更改关键内容，这样可以避免不必要的冲突和错误。

5. **版本历史记录**：OrcaStudio 应该有完整的版本控制功能，允许你查看、回滚到过去的版本，这对于问题排查和项目管理非常有用，减少了因错误而造成的返工成本。

6. **高效资产管理**：良好的资产管理可以帮助团队成员快速找到需要的资源，避免重复制作，节省时间。

7. **可视化剧本编辑**：对于游戏开发或动画制作，OrcaStudio 的剧本编辑器可能允许你在 3D 环境中直接编写和预览脚本，减少在不同软件之间切换的时间。

8. **集成第三方工具**：如果 OrcaStudio 能与你团队使用的其他创意工具（如 Maya、Unreal Engine 等）无缝对接，那么可以进一步提高效率，减少数据导入导出的困扰。

9. **培训和文档支持**：利用提供的培训资源和详细文档，可以帮助新成员更快地熟悉软件，从而快速参与项目。

要详细了解如何在 OrcaStudio 中实现这些功能，你应该查阅官方文档、教程或联系技术支持获取具体指导。


> Finished chain.


'OrcaStudio 是一个强大的工具，可以用于优化和降低成本的 3D 内容协作。虽然我无法提供具体的步骤（因为我是一个基于文本的模型，不能直接操作软件），但我可以给你一些一般性的建议来帮助你利用 OrcaStudio 实现这个目标：\n\n1. **统一工作流程**：OrcaStudio 支持多种 3D 格式和工具，允许团队成员使用他们熟悉的工具集。这减少了学习新软件的时间，从而提高了效率。\n\n2. **实时协作**：通过 OrcaStudio 的实时共享编辑功能，团队成员可以同时在同一个项目上工作，消除了版本控制的问题，减少因协同不当导致的错误和重做工作。\n\n3. **自动化任务**：OrcaStudio 可能包含一些脚本或自动化工具，可以自动执行重复的任务，比如资源打包、优化或者预处理，这将节省大量手动操作的时间。\n\n4. **精细权限管理**：通过设置不同成员的访问权限，你可以确保只有授权人员才能更改关键内容，这样可以避免不必要的冲突和错误。\n\n5. **版本历史记录**：OrcaStudio 应该有完整的版本控制功能，允许你查看、回滚到过去的版本，这对于问题排查和项目管理非常有用，减少了因错误而造成的返工成本。\n\n6. **高效资产管理**：良好的资产管理可以帮助团队成员快速找到需要的资源，避免重复制作，节省时间。\n\n7. **可视化剧本编辑**：对于游戏开发或动画制作，OrcaStudio 的剧本编辑器可能允许你在 3D 环境中直接编写和预览脚本，减少在不同软件之间切换的时间。\n\n8. **集成第三方工具**：如果 OrcaStudio 能与你团队使用的其他创意工具（如 Maya、Unreal Engine 等）无缝对接，那么可以进一步提高效率，减少数据导入导出的困扰。\n\n9. **培训和文档支持**：利用提供的培训资源和详细文档，可以帮助新成员更快地熟悉软件，从而快速参与项目。\n\n要详细了解如何在 OrcaStudio 中实现这些功能，你应该查阅官方文档、教程或联系技术支持获取具体指导。'

## 多跳向量存储推理

由于向量存储很容易作为代理中的工具使用，因此使用现有代理框架很容易使用依赖于向量存储的多跳问题的答案。

In [17]:
tools = [
    Tool(
        name="流浪地球 问答系统",
        func=state_of_union.run,
        description="当您需要回答有关最新流浪地球的问题时，非常有用。输入应该是一个完整的问题，不要引用之前对话中任何模糊的代词。",
    ),
    Tool(
        name="松应科技 OrcaStudio 问答系统",
        func=ruff.run,
        description="当您需要回答有关松应科技 OrcaStudio（一种3D内容协作平台）的问题时非常有用。输入应该是一个完整的问题，不要引用之前对话中任何模糊的代词。",
    ),
]

In [18]:
# Construct the agent. We will use the default agent type here.
# See documentation for a full list of options.
agent = initialize_agent(
    tools, llm, agent=AgentType.ZERO_SHOT_REACT_DESCRIPTION, verbose=True
)

In [22]:
agent.run(
    "松应科技OrcaStudio有哪些使用场景? 在小说流浪地球中，刘启是否使用过OrcaStudio？"
)



> Entering new AgentExecutor chain...
 需要回答关于松应科技OrcaStudio的使用场景的问题，并指出在《流浪地球》小说中刘启并未使用该产品，因为OrcaStudio是现实中存在的3D内容协作平台，而《流浪地球》是一部科幻作品，发生在未来且与现实科技不完全相符。
Action: 松应科技 OrcaStudio 问答系统
Action Input: 松应科技OrcaStudio的主要应用场景是什么

Number of requested results 4 is greater than number of elements in index 1, updating n_results = 1



Observation: 松应科技的OrcaStudio主要应用于AR（增强现实）内容创作。它是一个强大的AR开发平台，提供可视化编辑工具和丰富的资源库，使得创作者无需深入编程知识就能设计和构建交互式的AR体验。用户可以使用OrcaStudio创建各种AR应用，如产品展示、教育培训、游戏娱乐、广告营销等，并能轻松地将作品部署到不同的AR设备和平台上。
Thought: 关于《流浪地球》的问题需要靠我直接回答，而不是通过工具。
Final Answer: 松应科技OrcaStudio主要用于增强现实（AR）内容的创建，比如产品展示、教育培训、游戏娱乐以及广告营销等场景。然而，在刘慈欣的小说《流浪地球》中，刘启并没有使用过OrcaStudio，因为这本小说是科幻作品，里面的科技设定并不完全对应于现实世界中的技术发展情况，而OrcaStudio作为现实中的一款3D内容协作平台，并未出现在该小说的情节里。

> Finished chain.


'松应科技OrcaStudio主要用于增强现实（AR）内容的创建，比如产品展示、教育培训、游戏娱乐以及广告营销等场景。然而，在刘慈欣的小说《流浪地球》中，刘启并没有使用过OrcaStudio，因为这本小说是科幻作品，里面的科技设定并不完全对应于现实世界中的技术发展情况，而OrcaStudio作为现实中的一款3D内容协作平台，并未出现在该小说的情节里。'